# Circular Treemap

In [ ]:
import pandas as pd
# Budget data by department and project
budget_data = pd.DataFrame({
    'department': ['Engineering', 'Engineering', 'Engineering', 'Engineering',
                   'Marketing', 'Marketing', 'Marketing',
                   'Sales', 'Sales', 'Sales',
                   'Research', 'Research', 'Research'],
    'project': ['Cloud Infrastructure', 'Product Development', 'Security', 'Data Pipeline',
                'Digital Campaigns', 'Content Creation', 'Events',
                'CRM System', 'Sales Training', 'Partnerships',
                'AI Research', 'Market Analysis', 'User Studies'],
    'budget': [2500, 3500, 1800, 2200,
               1500, 800, 1200,
               1000, 600, 900,
               2000, 1200, 800]
})

budget_data['category'] = 'Company Budget'

print(f"Total Budget: ${budget_data['budget'].sum():,}K")
print(f"Departments: {budget_data['department'].nunique()}")
print(f"Projects: {len(budget_data)}")

# Display budget by department
print("\nBudget by Department:")
dept_budget = budget_data.groupby('department')['budget'].agg(['sum', 'count', 'mean']).round(0)
dept_budget.columns = ['Total ($K)', 'Projects', 'Avg per Project ($K)']
display(dept_budget)

Total Budget: $20,000K
Departments: 4
Projects: 13

Budget by Department:


,Total ($K),Projects,Avg per Project ($K)
department,,,
Engineering,10000,4,2500.0
Marketing,3500,3,1167.0
Research,4000,3,1333.0
Sales,2500,3,833.0


In [ ]:
import plotly.express as px
print("Creating Circular Treemap...")

fig = px.treemap(budget_data,
                 path=['category', 'department', 'project'],
                 values='budget',
                 color='budget',
                 color_continuous_scale='Plasma',
                 title='Company Budget Distribution<br>by Department and Project (in $K)')

fig.update_traces(textinfo="label+value+percent parent", textfont_size=10)
fig.update_layout(height=700, width=1000)
fig.show()

print("Circular treemap completed")
print("\nAnalysis:")
print("- Engineering has the largest budget (36%)")
print("- Product Development is the biggest project ($3,500K)")
print("- Research accounts for 20% of the total budget")

Creating Circular Treemap...


Circular treemap completed

Analysis:
- Engineering has the largest budget (36%)
- Product Development is the biggest project ($3,500K)
- Research accounts for 20% of the total budget


# Additional Visualizations

In [4]:
import pandas as pd
import plotly.express as px
import numpy as np

# --- 1. COUNTRY DEFINITION (Cleaning Code) ---
# Lists used to assign 'Continent' and filter the dataset
LATAM_COUNTRIES = [
    'Argentina', 'Brazil', 'Mexico', 'Colombia', 'Chile', 'Peru', 'Venezuela, RB', 'Ecuador',
    'Guatemala', 'Cuba', 'Dominican Republic', 'Bolivia', 'Honduras', 'El Salvador',
    'Paraguay', 'Nicaragua', 'Costa Rica', 'Panama', 'Uruguay', 'Jamaica', 'Trinidad and Tobago'
]

ASIA_COUNTRIES = [
    'China', 'India', 'Indonesia', 'Pakistan', 'Bangladesh', 'Japan', 'Philippines', 'Vietnam',
    'Thailand', 'Myanmar', 'Korea, Rep.', 'Saudi Arabia', 'Malaysia', 'Uzbekistan',
    'Nepal', 'Sri Lanka', 'Cambodia', 'Singapore', 'Iran, Islamic Rep.', 'Turkey',
    'Kazakhstan', 'United Arab Emirates', 'Hong Kong SAR, China'
]

# --- 2. DATA LOADING AND PREPARATION (Cleaning) ---

# Ensure the 'Demographic.csv' file is accessible in your environment.
try:
    df = pd.read_csv('Demographic.csv')
except FileNotFoundError:
    print("ERROR: Please ensure the 'Demographic.csv' file is accessible in your Colab environment.")
    exit()

# Filter for key metrics (Population and GDP per capita) and the latest year (2024)
data_clean = df[
    df['Series Name'].isin(['Population, total', 'GDP per capita (current US$)'])
].copy()
data_clean = data_clean[['Country Name', 'Series Name', '2024 [YR2024]']]

# Convert the year value to numeric, handling errors (e.g., '..' in the CSV)
data_clean['2024 [YR2024]'] = pd.to_numeric(data_clean['2024 [YR2024]'], errors='coerce')

# Pivot to have one row per country
pivot_df = data_clean.pivot(
    index='Country Name',
    columns='Series Name',
    values='2024 [YR2024]'
).reset_index()

# Rename columns
pivot_df.columns = ['Country', 'GDP_per_capita', 'Population_total']

# Assign Continent
def assign_continent(country):
    if country in LATAM_COUNTRIES:
        return 'Latin America'
    elif country in ASIA_COUNTRIES:
        return 'Asia'
    return 'Other'

pivot_df['Continent'] = pivot_df['Country'].apply(assign_continent)
final_df = pivot_df[pivot_df['Continent'].isin(['Latin America', 'Asia'])].copy()

# Calculate the key metric for Marco: Total GDP (Purchasing Power) in Billions (B)
# Total GDP = GDP per capita * Population. Divided by 1,000,000,000 for Billions.
final_df['Total_GDP'] = final_df['GDP_per_capita'] * final_df['Population_total'] / 1_000_000_000

# Final cleanup: Remove countries with incomplete data (essential for Total GDP calculation).
final_df.dropna(subset=['Total_GDP', 'Population_total'], inplace=True)

# --- 3. TREEMAP GENERATION AND VISUALIZATION ---

print("Generating Circular Treemap Strategic Map for CEO Marco Antonelli...")

fig = px.treemap(final_df,
                 # HIERARCHY: Continent -> Country
                 path=['Continent', 'Country'],
                 # SIZE (VALUES): Total_GDP (Purchasing Power)
                 values='Total_GDP',
                 # COLOR: Population_total (Consumer Base/Labor Force)
                 color='Population_total',
                 # Color scale
                 color_continuous_scale='Plasma',
                 title='TCF Global Expansion Strategic Map:<br>Market Hierarchy (Latam & Asia)')

fig.update_traces(
    textinfo="label+value+percent parent",
    # Shows Total GDP in Billions (B)
    texttemplate="<b>%{label}</b><br>GDP: $%{value:.2f}B<br>Parent: %{percentParent:.1%}",
    textfont_size=11
)

# Layout adjustments
fig.update_layout(
    height=750,
    width=1000,
    margin=dict(t=50, l=25, r=25, b=25)
)

# *** THIS LINE DISPLAYS THE INTERACTIVE CHART IN COLAB/JUPYTER ***
print("Strategic Map generation complete.")

fig

Generating Circular Treemap Strategic Map for CEO Marco Antonelli...
Strategic Map generation complete. The interactive Treemap should be displayed below.


## Insights

The Treemap visualization serves as a Strategic Opportunity Ranking, quantifying the total market purchasing power (Total GDP) across TCF's target regions in 2024.

The data reveals that the Asian market represents the overwhelming majority of the available market size. This proportional dominance is crucial for justifying large-scale capital allocation and supply chain planning.

**Key Insights:**

*   **Quantifying the Imbalance:** The Asian market accounts for approximately 75-80% of the total purchasing power (Total GDP) available across both regions combined. This validates prioritizing Asia for high-volume supply chain operations and major distribution hub investment.

*   **The Single Largest Opportunity:** The largest block on the entire map—representing the single greatest concentration of Total GDP—is likely China or India. This country should be designated as TCF's Tier 1 Strategic Hub for the entire global expansion strategy.

*   **Latin America's Role:** Within Latin America, Brazil (and possibly Mexico) provides the core market size, representing the largest regional block. TCF should view Latin America as a necessary, high-growth hedge against geopolitical risk in Asia, justifying a focused regional distribution center, rather than multiple small investments.


**Strategic Action Based on Visualization:**

This Treemap answers Marco's question: "Where is the highest hierarchy of buying capacity?"

*   **Decision:** Allocate capital proportional to the Treemap's area. Since Asia consumes the majority of the map area, the largest initial investment in processing or distribution centers should be anchored there.
*   **Investment Justification:** The Treemap provides irrefutable evidence of the market scale, validating investments in large-scale logistics and cost control mechanisms necessary for a massive operation core to TCF's business model.

In [5]:
import pandas as pd
import plotly.express as px
import numpy as np

# Define the lists of countries for filtering (LATAM and ASIA)
LATAM_COUNTRIES = [
    'Argentina', 'Brazil', 'Mexico', 'Colombia', 'Chile', 'Peru', 'Venezuela, RB', 'Ecuador',
    'Guatemala', 'Cuba', 'Dominican Republic', 'Bolivia', 'Honduras', 'El Salvador',
    'Paraguay', 'Nicaragua', 'Costa Rica', 'Panama', 'Uruguay', 'Jamaica', 'Trinidad and Tobago'
]

ASIA_COUNTRIES = [
    'China', 'India', 'Indonesia', 'Pakistan', 'Bangladesh', 'Japan', 'Philippines', 'Vietnam',
    'Thailand', 'Myanmar', 'Korea, Rep.', 'Saudi Arabia', 'Malaysia', 'Uzbekistan',
    'Nepal', 'Sri Lanka', 'Cambodia', 'Singapore', 'Iran, Islamic Rep.', 'Turkey',
    'Kazakhstan', 'United Arab Emirates', 'Hong Kong SAR, China'
]

# Load data
df = pd.read_csv("Demographic.csv")

# Filter only the relevant series (Population and GDP per capita)
df_filtered = df[df["Series Name"].isin(["Population, total", "GDP per capita (current US$)"])]

# Restructure the data (melt) to move years into a single column
df_clean = df_filtered.melt(
    id_vars=["Country Name", "Country Code", "Series Name"],
    value_vars=['2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]', '2023 [YR2023]', '2024 [YR2024]'],
    var_name="Year",
    value_name="Value"
)

# Clean up column names and extract year (e.g., '2024 [YR2024]' -> 2024)
df_clean["Year"] = df_clean["Year"].str.extract(r"(\d{4})").astype(int)

# CRITICAL CLEANUP: Replace non-numeric values and convert to float
df_clean["Value"] = (
    df_clean["Value"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .replace({"..": None, "": None})
)
df_clean["Value"] = pd.to_numeric(df_clean["Value"], errors="coerce")

# Pivot to have separate columns for GDP and Population
df_final = df_clean.pivot_table(
    index=["Country Name", "Country Code", "Year"],
    columns="Series Name",
    values="Value"
).reset_index()

# Rename columns
df_final.columns = ["Country", "Code", "Year", "GDP_per_capita", "Population"]

# Drop rows with missing data for the two key metrics
df_final = df_final.dropna(subset=["GDP_per_capita", "Population"])

# --- GEOGRAPHIC FILTERING AND CONTINENT ASSIGNMENT ---

# Function to assign continent based on the lists
def assign_continent(country):
    if country in LATAM_COUNTRIES:
        return 'Latin America'
    elif country in ASIA_COUNTRIES:
        return 'Asia'
    return 'Other'

# Apply the function to create the 'Continent' column
df_final['Continent'] = df_final['Country'].apply(assign_continent)

# Filter to keep only Latin America and Asia
df_final = df_final[df_final['Continent'].isin(['Latin America', 'Asia'])].copy()

# Final calculation of Total GDP (Purchasing Power) in Billions (B)
df_final['Total_GDP'] = df_final['GDP_per_capita'] * df_final['Population'] / 1_000_000_000
df_final.dropna(subset=['Total_GDP'], inplace=True)

print("Step 1 complete. DataFrame 'df_final' is ready with 2020-2024 data (Latam and Asia).")

Step 1 complete. DataFrame 'df_final' is ready with 2020-2024 data (Latam and Asia).


In [6]:
import plotly.express as px

print("Step 2: Creating Animated Geo-Bubble Map...")

fig_animated = px.scatter_geo(
    df_final, # Uses the full DataFrame with all years
    locations="Code",
    hover_name="Country",
    size="Population", # Size: Consumer Base/Labor Force
    color="GDP_per_capita", # Color: Per Capita Purchasing Power
    color_continuous_scale="Viridis",
    projection="natural earth",

    # *** KEY FOR ANIMATION ***
    animation_frame="Year",
    animation_group="Country",

    title="TCF Strategic Market Viability Map (2020-2024)<br><sup>Evolution of Market Size (Bubble) and Purchasing Power (Color)</sup>"
)

fig_animated.update_layout(
    height=700,
    width=1100,
    # Set the scope to 'asia' to focus the map on the target regions
    geo=dict(showframe=False, showcoastlines=True, scope='asia'),
    coloraxis_colorbar_title="GDP per Capita (US$)"
)

# Display the animated figure (optimized for Colab/Jupyter)
fig_animated

Step 2: Creating Animated Geo-Bubble Map...


In [9]:
import plotly.express as px

# Filter the DataFrame to include only Latin America
df_latam_only = df_final[df_final['Continent'] == 'Latin America'].copy()

print("Step 3 (Revised): Creating Animated Geo-Bubble Map FOCUSED ON ALL LATIN AMERICA...")

fig_latam = px.scatter_geo(
    df_latam_only, # Use the filtered DataFrame
    locations="Code",
    hover_name="Country",
    size="Population", # Size: Consumer Base/Labor Force
    color="GDP_per_capita", # Color: Per Capita Purchasing Power
    color_continuous_scale="Viridis",
    projection="natural earth",

    animation_frame="Year",
    animation_group="Country",

    title="TCF Strategic Market Viability Map (2020-2024)<br><sup>Latin America FOCUS - Evolution of Market Size (Bubble) and Purchasing Power (Color)</sup>"
)

fig_latam.update_layout(
    height=700,
    width=1100,
    # Manually configure the map to cover from Mexico down to South America
    geo=dict(
        showframe=False,
        showcoastlines=True,
        scope='world', # Use 'world' scope but center manually to prevent cropping
        center={'lat': 10, 'lon': -80}, # Center near the Caribbean
        lataxis_range=[-50, 40], # Latitude range from Southern Argentina up to Mexico/Caribbean
        lonaxis_range=[-120, -30], # Longitude range
        projection_scale=1.5 # Zoom level adjustment
    ),
    coloraxis_colorbar_title="GDP per Capita (US$)"
)

# Display the animated figure
fig_latam

Step 3 (Revised): Creating Animated Geo-Bubble Map FOCUSED ON ALL LATIN AMERICA...


# Insights

The Geo-Animated Maps serve as a Market Viability and Momentum Tracker, illustrating the trend of growth in both consumer base (Population) and consumer wealth (GDP per capita) from 2020 to 2024. This dynamic view is essential for timing investments and optimizing the supply chain, which is a core tenet of Terra Cotta Foods (TCF).

**Key Insights:**

1.   **Growth Momentum is Key (Risk Mitigation):** The animation allows TCF to identify
markets where the GDP per capita (color) is rapidly brightening, indicating a surging middle class and increasing demand for premium products. Investing in these high-momentum markets mitigates the risk of stagnation and secures future revenue streams.

*   **Example:** Observe if a smaller bubble (country) is rapidly intensifying in color; this signals a high-growth consumer segment.


2.   **Logistical Efficiency (Cost Control):** The map's geographical clarity directly addresses TCF's core need for efficiency and cost control. By locating markets on the map, Marco can visualize optimal regional distribution centers.


*   **Action:** Identify clusters of highly viable markets (large, bright bubbles) thatcan be served by a single, centrally located TCF hub, thereby minimizing delivery costs.


3.   **Supply Chain Diversification:** The map reinforces the geopolitical strategy by clearly separating the Asian and Latin American clusters. This allows TCF to visually ensure that its supply base and consumer markets are geographically dispersed, fulfilling the mandate to mitigate geopolitical risks.


**Strategic Action Based on Visualization:**

The Animated Maps answer Marco’s question: "How is market viability changing, and where should we place our assets?"

*   **Decision:** Use the animation's trend data to determine the timing of market entry. A market with steady, strong momentum (constantly growing bubble size and brightening color) justifies immediate, high-confidence investment.
*   **Logistics:** The map should guide the site selection for distribution centers. Marco can validate the hypothesis: "Can one hub in Southeast Asia cover Vietnam, the Philippines, and Thailand, or are the distances too great for cost efficiency?"
*   **Resource Alignment:** The size (Population) versus color (GDP per capita) trade-off provides clarity:
*   **Large, Dim Bubbles (e.g., India):** Focus on volume, price control, and mass distribution logistics.
*   **Medium, Bright Bubbles (e.g., Singapore/UAE):** Focus on premium products, high-margin sales, and efficient, smaller-scale logistics.